In [140]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [152]:
def match_id_to_datetime(match_id: str) -> datetime:
    match_id = match_id.removeprefix("Match-")
    timestamp = datetime.strptime(match_id, "%Y-%m-%d_%H-%M-%S-%f")
    # Round to nearest second
    timestamp = timestamp.replace(microsecond=round(timestamp.microsecond, -6))
    return timestamp

In [155]:
def get_dfs_from_sheet(xlsx_path: Path, sheet_name: str):
    # Match/round info
    match_id = xlsx_path.stem
    round_num = int(sheet_name.split(" ")[-1])
    round_id = f"{match_id}{round_num}".removeprefix("Match-").replace("_", "").replace("-", "")

    # Player stats
    player_df = pd.read_excel(xlsx_path, sheet_name=sheet_name, skiprows=1)
    player_df = player_df.iloc[: player_df.Player.isnull().values.argmax()].copy()
    player_df.drop(columns={"Unnamed: 9", "Player 1", "Player 2", "Time", "Hs%"}, inplace=True)
    player_df.insert(0, "Match ID", match_id)
    player_df.insert(1, "Round Number", round_num)
    player_df.insert(2, "Round ID", round_id)

    # Round stats
    round_df = pd.read_excel(xlsx_path, sheet_name=sheet_name, skiprows=len(player_df) + 4)
    round_df = round_df.iloc[: round_df["Name"].isnull().values.argmax()].copy().fillna("")
    round_df.drop(columns=[f"Unnamed: {i}" for i in range(3, 13)], inplace=True)
    actual_round_info = {}
    for _, row in round_df.iterrows():
        if row["Name"] == "Winning team":
            winning_team, winning_team_idx = row["Value"].split(" ")
            winning_team_idx = int(winning_team_idx[1:-1])
            actual_round_info["Winning Team"], actual_round_info["Winning Team Index"] = winning_team, winning_team_idx
        else:
            actual_round_info[row["Name"].title()] = row["Value"]
        if row["Time"]:
            actual_round_info["Opening Kill Time"] = row["Time"]
    actual_round_df = pd.DataFrame([actual_round_info])
    actual_round_df.insert(0, "Match ID", match_id)
    actual_round_df.insert(1, "Round Number", round_num)
    actual_round_df.insert(2, "Round ID", round_id)
    actual_round_df.insert(3, "Match Timestamp", match_id_to_datetime(match_id))

    # Add trades to player stats
    # TODO: This section of the sheet is actually broken, calculate trades myself
    player_df["Trades"] = 0
    trades_df = pd.read_excel(xlsx_path, sheet_name=sheet_name, skiprows=1, usecols=["Player 1", "Player 2", "Time"]).dropna()
    for _, row in trades_df.iterrows():
        player_name = row["Player 1"]
        for _, player_row in player_df.iterrows():
            if player_row["Player"] == player_name:
                player_df.at[player_row.name, "Trades"] += 1
                break

    return player_df, actual_round_df

In [157]:
player_df = pd.DataFrame()
round_df = pd.DataFrame()

file_path = Path("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/2_decoded_replays/Match-2024-04-14_17-41-21-37.xlsx")
with pd.ExcelFile(file_path) as xlsx:
    for sheet_name in xlsx.sheet_names:
        if sheet_name.startswith("Match"):
            continue

        sheet_player_df, sheet_round_df = get_dfs_from_sheet(file_path, sheet_name)
        # print(sheet_player_df)
        print(sheet_round_df)

        break

                       Match ID  Round Number           Round ID  \
0  Match-2024-04-14_17-41-21-37             1  20240414174121371   

      Match Timestamp                           Site Winning Team  \
0 2024-04-14 17:41:21  B Laundry Room, B Supply Room     Kennesaw   

   Winning Team Index Win Condition Opening Kill Opening Kill Time  \
0                   1          Time   meinheiden              2:19   

  Opening Death Planted At Defused At  
0         Brij-                        
